In [2]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/ani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

In [ ]:
data = pd.read_csv(r'../fake_or_real_news.csv')

In [5]:
data = data.drop(['author', 'text'], axis = 1)

In [8]:
data = data.iloc[:, :2]
data.head()

,title,label
0,You Can Smell Hillary’s Fear,FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,FAKE
2,Kerry to go to Paris in gesture of sympathy,REAL
3,Bernie supporters on Twitter erupt in anger ag...,FAKE
4,The Battle of New York: Why This Primary Matters,REAL


In [9]:
data = data.dropna()

In [10]:
data.loc[data.label != 'REAL', 'label'] = 1
data.loc[data.label == 'REAL', 'label'] = 0

In [11]:
y = data.iloc[:, -1:].values
y = [i for sub in y for i in sub]

In [15]:
x = data['title']
x

0                             You Can Smell Hillary’s Fear
1        Watch The Exact Moment Paul Ryan Committed Pol...
2              Kerry to go to Paris in gesture of sympathy
3        Bernie supporters on Twitter erupt in anger ag...
4         The Battle of New York: Why This Primary Matters
                               ...                        
70795    Tesla Earnings Smash Expectations After Dramat...
70796     Rules For Rulers (Or How The World Really Works)
70797    Fact Check: Trump Is Right that Clinton Might ...
70798    Caught On Tape: ISIS Destroys Iraqi Abrams Wit...
70799    ObamaCare Architect Admits "The Law Is Working...
Name: title, Length: 65046, dtype: object

In [13]:
def stemm(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    text = [w for w in text if not w in stop_words]
    text = [stemmer.stem(word) for word in text]
    text = " ".join(text)
    return text

In [16]:
x = x.apply(stemm)
x

0                                       smell hillari fear
1        watch exact moment paul ryan commit polit suic...
2                            kerri go pari gestur sympathi
3           berni support twitter erupt anger dnc tri warn
4                            battl new york primari matter
                               ...                        
70795    tesla earn smash expect dramat chang report me...
70796                         rule ruler world realli work
70797         fact check trump right clinton might caus ww
70798    caught tape isi destroy iraqi abram anti tank ...
70799    obamacar architect admit law work design premi...
Name: title, Length: 65046, dtype: object

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression

In [18]:
c_vec = CountVectorizer(max_features=5000)
tfidf_vec = TfidfVectorizer(max_features=5000)
lr_model = LogisticRegression()

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1)

In [20]:
x_train = tfidf_vec.fit_transform(x_train).toarray()
x_test = tfidf_vec.transform(x_test).toarray()

In [21]:
def logReg(x, y):
  lr_model.fit(x_train, y_train)
  y_trainPred = lr_model.predict(x_train)
  y_testPred = lr_model.predict(x_test)
  print("Accuracy of training = ", accuracy_score(y_trainPred, y_train))
  print('Training accuracy\n')
  print(classification_report(y_train, y_trainPred))
  print("Accuracy of Testing = ", accuracy_score(y_testPred, y_test))
  print('Testing Results\n')
  print(classification_report(y_test, y_testPred))

In [22]:
logReg(x_train, y_train)

Accuracy of training =  0.9568952263817357
Training accuracy

              precision    recall  f1-score   support

           0       0.85      0.12      0.22      2497
           1       0.96      1.00      0.98     49539

    accuracy                           0.96     52036
   macro avg       0.90      0.56      0.60     52036
weighted avg       0.95      0.96      0.94     52036

Accuracy of Testing =  0.9518063028439662
Testing Results

              precision    recall  f1-score   support

           0       0.71      0.09      0.17       664
           1       0.95      1.00      0.98     12346

    accuracy                           0.95     13010
   macro avg       0.83      0.55      0.57     13010
weighted avg       0.94      0.95      0.93     13010

